# Dimer Mosaic Geometric Construction

This notebook implements the hardikmakhija dimer mosaic approach:
- Uses specific geometric constants for interlocking: dx=0.462, dy=0.522, ox=0.231
- Builds configurations from scratch using optimal geometric patterns
- This is a CONSTRUCTIVE approach, not optimization

In [1]:
import numpy as np
import pandas as pd
import math
import time
import os
from numba import njit
from shapely import affinity
from shapely.geometry import Polygon
from shapely.strtree import STRtree

print("Starting dimer mosaic approach...")

Starting dimer mosaic approach...


In [2]:
# Tree shape constants
TRUNK_W = 0.15
TRUNK_H = 0.2
BASE_W = 0.7
MID_W = 0.4
TOP_W = 0.25
TIP_Y = 0.8
TIER_1_Y = 0.5
TIER_2_Y = 0.25
BASE_Y = 0.0
TRUNK_BOTTOM_Y = -TRUNK_H

def get_tree_poly(x, y, deg):
    """Create a Shapely polygon for a tree at given position and angle."""
    coords = [
        (0.0, TIP_Y),
        (TOP_W / 2.0, TIER_1_Y),
        (TOP_W / 4.0, TIER_1_Y),
        (MID_W / 2.0, TIER_2_Y),
        (MID_W / 4.0, TIER_2_Y),
        (BASE_W / 2.0, BASE_Y),
        (TRUNK_W / 2.0, BASE_Y),
        (TRUNK_W / 2.0, TRUNK_BOTTOM_Y),
        (-TRUNK_W / 2.0, TRUNK_BOTTOM_Y),
        (-TRUNK_W / 2.0, BASE_Y),
        (-BASE_W / 2.0, BASE_Y),
        (-MID_W / 4.0, TIER_2_Y),
        (-MID_W / 2.0, TIER_2_Y),
        (-TOP_W / 4.0, TIER_1_Y),
        (-TOP_W / 2.0, TIER_1_Y),
    ]
    poly = Polygon(coords)
    return affinity.translate(affinity.rotate(poly, deg, origin=(0, 0)), x, y)

def calculate_side_length(trees):
    """Calculate bounding square side length."""
    if not trees:
        return 0.0
    all_polys = [t['poly'] for t in trees]
    from shapely.ops import unary_union
    bounds = unary_union(all_polys).bounds
    width = bounds[2] - bounds[0]
    height = bounds[3] - bounds[1]
    return max(width, height)

def calculate_score(trees, n):
    """Calculate score = side^2 / n."""
    side = calculate_side_length(trees)
    return side * side / n

In [3]:
# Dimer mosaic packing function from hardikmakhija kernel
def pack_optimized(n):
    """Pack n trees using dimer mosaic approach with aggressive interlocking."""
    # Aggressive interlocking constants from hardikmakhija kernel
    dx, dy = 0.462, 0.522
    ox = 0.231 
    buffer = 0.0
    
    while True:
        trees = []
        cols = int(np.ceil(np.sqrt(n)))
        for i in range(n):
            r, c = divmod(i, cols)
            flip = (i % 2 != 0)
            tx = c * (dx + buffer) + (ox if flip else 0)
            ty = r * (dy + buffer)
            ang = 180 if flip else 0
            trees.append({'x': tx, 'y': ty, 'deg': ang, 'poly': get_tree_poly(tx, ty, ang)})
        
        # Check for overlaps using STRtree for efficiency
        polys = [t['poly'] for t in trees]
        idx_tree = STRtree(polys)
        overlap = False
        for j, p in enumerate(polys):
            for k in idx_tree.query(p):
                if k != j and p.intersects(polys[k]):
                    overlap = True
                    break
            if overlap:
                break
        
        if not overlap:
            return trees
        buffer += 0.005

print("Dimer mosaic packing function defined.")

Dimer mosaic packing function defined.


In [4]:
# Load baseline for comparison
def load_baseline():
    baseline_path = '/home/code/external_data/saspav_latest/santa-2025.csv'
    df = pd.read_csv(baseline_path)
    for col in ['x', 'y', 'deg']:
        if df[col].dtype == object:
            df[col] = df[col].astype(str).str.replace('s', '').astype(float)
    
    baseline = {}
    for n in range(1, 201):
        prefix = f"{n:03d}_"
        group = df[df["id"].str.startswith(prefix)].sort_values("id")
        if len(group) == n:
            trees = []
            for _, row in group.iterrows():
                trees.append({
                    'x': row['x'],
                    'y': row['y'],
                    'deg': row['deg'],
                    'poly': get_tree_poly(row['x'], row['y'], row['deg'])
                })
            baseline[n] = trees
    return baseline

print("Loading baseline...")
baseline = load_baseline()
print(f"Loaded baseline with {len(baseline)} configurations")

# Calculate baseline scores
baseline_scores = {}
for n in range(1, 201):
    if n in baseline:
        baseline_scores[n] = calculate_score(baseline[n], n)

baseline_total = sum(baseline_scores.values())
print(f"Baseline total score: {baseline_total:.6f}")

Loading baseline...


Loaded baseline with 200 configurations


Baseline total score: 70.659958


In [5]:
# Generate dimer mosaic configurations for all N
print("\nGenerating dimer mosaic configurations...")
t0 = time.time()

dimer_configs = {}
dimer_scores = {}

for n in range(1, 201):
    trees = pack_optimized(n)
    dimer_configs[n] = trees
    dimer_scores[n] = calculate_score(trees, n)
    
    if n % 50 == 0:
        print(f"  N={n}: dimer={dimer_scores[n]:.6f}, baseline={baseline_scores[n]:.6f}")

print(f"\nGeneration completed in {time.time() - t0:.1f}s")


Generating dimer mosaic configurations...


  N=50: dimer=1.158851, baseline=0.360753


  N=100: dimer=1.127419, baseline=0.345531


  N=150: dimer=1.043668, baseline=0.337064


  N=200: dimer=1.050670, baseline=0.337564

Generation completed in 126.9s


In [6]:
# Compare dimer mosaic with baseline
print("\nComparing dimer mosaic with baseline:")

dimer_total = sum(dimer_scores.values())
print(f"Dimer mosaic total score: {dimer_total:.6f}")
print(f"Baseline total score: {baseline_total:.6f}")
print(f"Difference: {dimer_total - baseline_total:+.6f}")

# Count improvements
improvements = []
for n in range(1, 201):
    if dimer_scores[n] < baseline_scores[n]:
        improvements.append((n, baseline_scores[n], dimer_scores[n], baseline_scores[n] - dimer_scores[n]))

print(f"\nN values where dimer is better: {len(improvements)}")
if improvements:
    print("Top 10 improvements:")
    for n, baseline_s, dimer_s, imp in sorted(improvements, key=lambda x: -x[3])[:10]:
        print(f"  N={n}: baseline={baseline_s:.6f}, dimer={dimer_s:.6f}, improvement={imp:.6f}")


Comparing dimer mosaic with baseline:
Dimer mosaic total score: 231.356361
Baseline total score: 70.659958
Difference: +160.696403

N values where dimer is better: 0


In [7]:
# Create best-of-both ensemble
print("\nCreating best-of-both ensemble...")

best_configs = {}
best_scores = {}

for n in range(1, 201):
    if dimer_scores[n] < baseline_scores[n]:
        best_configs[n] = dimer_configs[n]
        best_scores[n] = dimer_scores[n]
    else:
        best_configs[n] = baseline[n]
        best_scores[n] = baseline_scores[n]

best_total = sum(best_scores.values())
print(f"Best-of-both total score: {best_total:.6f}")
print(f"Improvement over baseline: {baseline_total - best_total:.6f}")


Creating best-of-both ensemble...
Best-of-both total score: 70.659958
Improvement over baseline: 0.000000


In [8]:
# Save submission
print("\nSaving submission...")
os.makedirs('/home/submission', exist_ok=True)

rows = []
for n in range(1, 201):
    config = best_configs[n]
    for i, t in enumerate(config):
        rows.append({
            'id': f'{n:03d}_{i:03d}',
            'x': t['x'],
            'y': t['y'],
            'angle': t['deg']
        })

df = pd.DataFrame(rows)
df.to_csv('/home/submission/submission.csv', index=False)
print(f"Saved to /home/submission/submission.csv")
print(f"Total rows: {len(df)}")
print(f"\nFinal score: {best_total:.9f}")


Saving submission...
Saved to /home/submission/submission.csv
Total rows: 20100

Final score: 70.659958322
